### 12.03 Implement and ISBN cache

The International Standard Book Number (ISBN) is a unique commercial book identifier.  It is a string of length 10.  The first 9 characters are digits; the last character is a check character.  The check character is the sum of the first 9 digits, mod 11, with 10 represented by 'X'.  

Create a cache for looking up prices of books identified by their ISBN.  You implement lookup, insert, and remove methods.  Use the Least Recently Used (LRU) policy for cache eviction.  If an ISBN is already present, insert should not change the price, but it should update that entry to be the most recently used entry.

***Hint***: Amortize the cost of deletion.  Alternatively, use an auxiliary data structure.

### Remarks:

If we keep a counter of unique timestamps, then perhaps we can maintain an auxiliary data structure of sorted timestamps corresponding to isbns in our cache.  Our cache will be implemented as two dicts.

In [1]:
import bisect

class Cache:

    
    def __init__(self, size=5):
        self.size = size # size of cache
        self.t = 0  #timestamp
        self.i2t = {} #isbn to timestampe
        self.t2i = {} #timestamp to isbn
        self.timestamps = [] # sorted timestamps
    
    def __repr__(self):
        return "t: {}, i2t: {}, t2i: {}, timestamps: {}".format(
            self.t, self.i2t, self.t2i, self.timestamps)
        
    def lookup(self, isbn):
        print("isbn {}: {}found".format(isbn, "" if isbn in self.i2t else "not " ))
        return isbn in self.i2t
       
    def insert(self, isbn):
        if isbn in self.i2t:
            print("Updating timestamp")
            old_t = self.i2t[isbn]
            a = bisect.bisect_left(self.timestamps, old_t)

            self.timestamps = self.timestamps[:a] + self.timestamps[a + 1:]
            del self.t2i[old_t]
        self.i2t[isbn] = self.t
        self.t2i[self.t] = isbn 
        a = bisect.bisect_left(self.timestamps, self.t)
        self.timestamps = self.timestamps[:a] + [self.t] + self.timestamps[a:]
        self.t += 1
        if len(self.i2t) > self.size:
            old_t = self.timestamps.pop(0)
            old_i = self.t2i[old_t]
            print("{} removed from cache".format(old_i))
            del self.t2i[old_t]
            del self.i2t[old_i]
    
    def remove(self, isbn):
        print("Deleting")
        old_t = self.i2t[isbn]
        a = bisect.bisect_left(self.timestamps, old_t)
        self.timestamps = self.timestamps[:a] + self.timestamps[a + 1:]
        del self.t2i[old_t]
        del self.i2t[isbn]

cache = Cache(size=3)
cache.lookup("A")
cache.insert("A")
cache.lookup("A")
cache.insert("C")
cache.insert("C")
cache.insert("B")
cache.insert("A")
cache.remove("B")
cache.insert("D")
cache.insert("F")
print(cache)

isbn A: not found
isbn A: found
Updating timestamp
Updating timestamp
Deleting
C removed from cache
t: 7, i2t: {'A': 4, 'D': 5, 'F': 6}, t2i: {4: 'A', 5: 'D', 6: 'F'}, timestamps: [4, 5, 6]
